# Model Training - Alpha-Hunter

本notebook用于训练和比较不同的预测模型：
- Transformer
- Ridge Regression
- Random Forest
- MLP

采用rolling window方法进行时间序列交叉验证。


In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.data_loader import SequenceDataLoader
from src.models import TransformerPredictor, RidgePredictor, RandomForestPredictor, MLPPredictor
from src.trainer import RollingWindowTrainer
from src.evaluator import PerformanceEvaluator
from src.config import Config
from src.utils import set_random_seed, check_gpu_availability

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', 20)
set_random_seed(42)
print("Libraries loaded successfully!")


## 1. 配置和数据加载


In [ ]:
# Configuration
config = Config()

# 检查GPU可用性
device = check_gpu_availability()
config.transformer.device = device
config.mlp.device = device

# 数据路径
PCA_PATH = '../feature/pca_feature_store.csv'
OUTPUT_DIR = Path('../results')
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"Device: {device}")
print(f"PCA Path: {PCA_PATH}")
print(f"Output Directory: {OUTPUT_DIR}")


In [ ]:
# 加载数据
data_loader = SequenceDataLoader(pca_path=PCA_PATH, sequence_length=12, forward_fill_limit=3)
stats = data_loader.get_statistics()
print("\\nDataset Statistics:")
print(f"  Number of dates: {stats['n_dates']}")
print(f"  Number of features: {stats['n_features']}")
print(f"  Date range: {stats['date_range']}")
data_loader.df.head()


## 2. 训练模型

提示：完整训练可能需要较长时间。可以先用小样本测试，或使用命令行脚本 `train.py`。

示例：训练Transformer模型


In [ ]:
# 创建Transformer训练器
def create_transformer():
    return TransformerPredictor(
        input_dim=stats['n_features'],
        d_model=64, nhead=4, num_layers=2,
        epochs=20,  # 减少epochs用于快速测试
        device=device
    )

trainer = RollingWindowTrainer(
    data_loader=data_loader,
    model_factory=create_transformer,
    output_dir=OUTPUT_DIR / 'transformer'
)

# 训练和预测（这一步会运行较长时间）
# predictions = trainer.train_and_predict(verbose=True)
# predictions.head()


## 3. 加载已有结果进行分析

如果已经使用 `train.py` 训练了模型，可以加载结果进行分析：


In [ ]:
# 加载预测结果（如果存在）
import glob

prediction_files = glob.glob(str(OUTPUT_DIR / '*/predictions_*.csv'))
if prediction_files:
    print(f"找到 {len(prediction_files)} 个预测文件")
    # 加载最新的预测
    latest_file = max(prediction_files, key=lambda x: Path(x).stat().st_mtime)
    predictions = pd.read_csv(latest_file)
    predictions['date'] = pd.to_datetime(predictions['date'])
    print(f"加载: {latest_file}")
    print(f"预测数量: {len(predictions)}")
    predictions.head()
else:
    print("未找到预测文件。请先运行: python ../train.py --model transformer")


## 总结

本notebook提供了模型训练的框架。由于完整训练耗时较长，建议：

1. **快速测试**：修改上面代码减少epochs和train_window
2. **完整训练**：使用命令行脚本
   ```bash
   cd ..
   python train.py --model transformer --verbose
   python train.py --model all  # 训练所有模型
   ```
3. **结果分析**：使用 `02_backtesting.ipynb` 进行详细分析

下一步：前往 `02_backtesting.ipynb` 进行回测分析。
